<a href="https://colab.research.google.com/github/iagoslopes/DSM_PLN/blob/master/Aula8_ML_Classificacao/%5BPLN%5D_Aula_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 08 - Introdução a Machine Learning para PLN
**Objetivo da aula:**
*  Introduzir os fundamentos de aprendizado de máquina aplicados ao PLN, focando na classificação de texto e regressão, utilizando algoritmos como Naive Bayes e SVM.

## Exemplo 01 - Aplicação do modelo de Naives em um texto

In [ ]:
# Passo 1 - Criar o Corpus
corpus = [
    ("Eu amo PLN","positivo"),
    ("Eu odeio bugs","negativo"),
    ("Amo resolver problemas","positivo"),
    ("Odeio erros","negativo"),
    ("Bugs legais","positivo"),
    ("Resolver","positivo"),
    ("Eu não","negativo")
]

# Passo 2 Pre-Processar o Texto
import re
from collections import defaultdict, Counter

def preprocess_text(text):
  return re.findall(r'\b\w+\b',text.lower())

processed_corpus = [(preprocess_text(text), label) for text, label in corpus]
print(processed_corpus)

# Passo 3 - Calculando probabilidades
  # Este passo implementa um modelo baseado em probabilidades, como Naive Bayes.
class_counts = Counter()  # Conta o número de exemplos em cada classe (positivo ou negativo).
word_counts = defaultdict(Counter)  # Conta a frequência de cada palavra em cada classe.
total_words = defaultdict(int)  # Mantém o total de palavras em cada classe.

for words, label in processed_corpus:
  class_counts[label] += 1 # Incrementa a contagem de exemplos para a classe atual.
  for word in words:
    word_counts[label][word] += 1 # Incrementa a contagem da palavra na classe correspondente.
    total_words[label] # Incrementa o total de palavras na classe correspondente.

# Calcula a probabilidade a priori (probabilidade de cada classe independente do texto)
  # Fórmula: P(classe) = (número de exemplos na classe) / (total de exemplos)
total_examples = sum(class_counts.values())
prior_probabilities = {cls: count / total_examples for cls, count in class_counts.items()}

# Define uma função para calcular a probabilidade condicional de uma palavra dada a classe
  # Fórmula com suavização de Laplace: P(palavra | classe) = (contagem(palavra) + alpha) / (total de palavras + alpha * número de palavras únicas)
def conditional_probability(word, label, alpha=1):
  return (word_counts[label][word] + alpha) / (total_words[label] + alpha * len(word_counts[label]))

# Passo 4 - Classificar um novo Texto
  # Este passo implementa a classificação de um texto com base nas probabilidades.

# Define uma função para prever a classe de um novo texto.
def predict(text):
    words = preprocess_text(text)  # Pré-processa o texto de entrada.
    probabilities = {}  # Dicionário para armazenar as probabilidades de cada classe.

    # Para cada classe, calcula a probabilidade do texto pertencer a ela.
    for label in class_counts.keys():
        probabilities[label] = prior_probabilities[label]  # Começa com a probabilidade a priori da classe.
        for word in words:
            probabilities[label] *= conditional_probability(word, label)  # Multiplica pela probabilidade condicional de cada palavra.

    # Retorna a classe com maior probabilidade e as probabilidades calculadas.
    return max(probabilities, key=probabilities.get), probabilities

### Passo 5 realizar o teste com novo texto
* Comando:
  * print(f'Texto: "{novo_texto}"') - imprime o valor do novo texto informado na linha 2 da celula abaixo

In [ ]:
# Passo 5 - Teste com um novo texto
  # Testa a função de predição com um exemplo.
novo_texto = "Eu amo resolver bugs"
classe, probs = predict(novo_texto)  # Faz a predição do texto.

# Exibe os resultados.
print(f'Texto: "{novo_texto}"')
print(f'Classe prevista: `{classe}`')
print(f'Probabilidades:')
for label, prob in probs.items():
    print(f" {label}: {prob}")

## Exemplo 2 - Classificação do SVM (Support Vector Machines)

In [ ]:
!pip install scikit-learn

In [ ]:
# Passo 1 Importação das bibliotecas a serem utilizadas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Passo 2 Dados Exemplo
corpus = [
    "Eu amo PLN", "Eu odeio bugs", "Eu amo resolver problemas",
    "Odeio erros","Amo programação","Não gosto de falhas"
]
classes = ["positivo","negativo","positivo","negativo","positivo","negativo"]

# Passo 3 - Pré-processamento e vetorização
  # TfidfVectorizer converte texto em uma matriz numérica baseada na frequência inversa do termo (TF-IDF).
  # Cada palavra é transformada em um valor que reflete sua importância relativa no corpus.
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)  # Ajusta o vetorizador ao corpus e transforma o texto em números.
y = classes  # As classes correspondentes (rótulos).

# Passo 4 - Dividir os dados e treinar o modelo
  # Divide os dados em conjuntos de treino (70%) e teste (30%).
  # random_state garante que a divisão seja reproduzível.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Cria um modelo de SVM com kernel linear (linha de decisão linear entre as classes).
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)  # Treina o modelo com os dados de treino.

# Passo 5 - Avaliar o modelo
  # Faz previsões no conjunto de teste usando o modelo treinado.
y_pred = svm_model.predict(X_test)

# Gera um relatório de classificação contendo precisão, recall e F1-score para cada classe.
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00         1
    positivo       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



## Exemplo 3 - Comparando os Classificadores com Scikit-learn

In [ ]:
# 1.Importar Bibliotecas
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report

  # Baixar o dataset de exemplo
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

# 2. Preparação dos dados

  # Coleta de textos e classes
    # Combina as palavras de cada documento em uma única string, associando-as à sua categoria (positiva ou negativa).
documents = [(" ".join(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

  # Separar textos e rótulos
    # Divide os documentos em textos (as resenhas) e rótulos (positivo/negativo).
texts, labels = zip(*documents)

  # Transformar rótulos (positivo/negativo) em 0 e 1
    # Converte as categorias textuais em valores numéricos para facilitar o uso nos modelos.
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)  # "positivo" se torna 1 e "negativo" se torna 0.

  # Dividir dados em treino e teste
    # Divide os textos e rótulos em conjuntos de treino (70%) e teste (30%).
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.3, random_state=42)

# 3. Representação do texto com TF-IDF
  # Criar o vetorizador TF-IDF
    # Limita o vocabulário a 5.000 palavras mais frequentes para reduzir a dimensionalidade.
vectorizer = TfidfVectorizer(max_features=5000)

  # Ajustar e transformar os textos
    # Ajusta o vetorizador aos textos de treino e os transforma em matrizes numéricas (TF-IDF).
X_train = vectorizer.fit_transform(texts_train)
  # Transforma os textos de teste com base no vetorizador ajustado nos dados de treino.
X_test = vectorizer.transform(texts_test)

# 4. Treinar os modelos
  # Treinamento do Naive Bayes
    # Ajusta o modelo Naive Bayes Multinomial aos dados de treino.
nb_model = MultinomialNB()
nb_model.fit(X_train, labels_train)

  # Treinamento do SVM
    # Ajusta o modelo SVM com um kernel linear aos dados de treino.
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, labels_train)

  # Predição
    # Usa o modelo treinado para prever os rótulos dos dados de teste.
nb_predictions = nb_model.predict(X_test)
svm_predictions = svm_model.predict(X_test)

# 5. Avaliação
  # Avaliação do Naive Bayes
    # Exibe métricas de desempenho como precisão, recall e F1-score para o modelo Naive Bayes.
print("Naive Bayes Performance:")
print(classification_report(labels_test, nb_predictions, target_names=label_encoder.classes_))

  # Avaliação do SVM
    # Exibe métricas de desempenho como precisão, recall e F1-score para o modelo SVM.
print("SVM Performance:")
print(classification_report(labels_test, svm_predictions, target_names=label_encoder.classes_))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


Naive Bayes Performance:
              precision    recall  f1-score   support

         neg       0.79      0.84      0.81       302
         pos       0.82      0.77      0.80       298

    accuracy                           0.80       600
   macro avg       0.80      0.80      0.80       600
weighted avg       0.80      0.80      0.80       600

SVM Performance:
              precision    recall  f1-score   support

         neg       0.82      0.80      0.81       302
         pos       0.81      0.82      0.81       298

    accuracy                           0.81       600
   macro avg       0.81      0.81      0.81       600
weighted avg       0.81      0.81      0.81       600

